In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from keras.applications import imagenet_utils
from tensorflow.keras import layers
from tensorflow import keras
!pip install tensorflow-addons
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
tfds.disable_progress_bar()

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 30.8 MB/s eta 0:00:00
2.12.0


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
IMG_SIZE = 256
PATCH_SIZE = 4  # 2x2, for the Transformer blocks
EPOCHS = 100
BATCH_SIZE = 64
num_classes = 4
learning_rate = 0.001
label_smoothing_factor = 0.1
expansion_factor = 2  # expansion factor for the MobileNetV2 blocks.

In [4]:
def conv_block(x, filters=16, kernel_size=3, strides=2):
    conv_layer = layers.Conv2D(
        filters, kernel_size, strides=strides, activation=tf.nn.swish, padding="same"
    )
    return conv_layer(x)

def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
    m = layers.Conv2D(expanded_channels, 1, padding="same", use_bias=False)(x)
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    if strides == 2:
        m = layers.ZeroPadding2D(padding=imagenet_utils.correct_pad(m, 3))(m)
    m = layers.DepthwiseConv2D(
        3, strides=strides, padding="same" if strides == 1 else "valid", use_bias=False
    )(m)
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    m = layers.Conv2D(output_channels, 1, padding="same", use_bias=False)(m)
    m = layers.BatchNormalization()(m)

    if tf.math.equal(x.shape[-1], output_channels) and strides == 1:
        return layers.Add()([m, x])
    return m

def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.swish)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, x])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=[x.shape[-1] * 2, x.shape[-1]], dropout_rate=0.1,)
        # Skip connection 2.
        x = layers.Add()([x3, x2])
    return x

def mobilevit_block(x, num_blocks, projection_dim, strides=1):
    # Local projection with convolutions.
    local_features = conv_block(x, filters=projection_dim, strides=strides)
    local_features = conv_block(
        local_features, filters=projection_dim, kernel_size=1, strides=strides
    )

    # Unfold into patches and then pass through Transformers.
    num_patches = int((local_features.shape[1] * local_features.shape[2]) / PATCH_SIZE)
    non_overlapping_patches = layers.Reshape((PATCH_SIZE, num_patches, projection_dim))(
        local_features
    )
    global_features = transformer_block(
        non_overlapping_patches, num_blocks, projection_dim
    )

    # Fold into conv-like feature-maps.
    folded_feature_map = layers.Reshape((*local_features.shape[1:-1], projection_dim))(
        global_features
    )

    # Apply point-wise conv -> concatenate with the input features.
    folded_feature_map = conv_block(
        folded_feature_map, filters=x.shape[-1], kernel_size=1, strides=strides
    )
    local_global_features = layers.Concatenate(axis=-1)([x, folded_feature_map])

    # Fuse the local and global features using a convoluion layer.
    local_global_features = conv_block(
        local_global_features, filters=projection_dim, strides=strides
    )

    return local_global_features

def create_mobilevit(num_classes=4):
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Initial conv-stem -> MV2 block.
    x = conv_block(x, filters=16)
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=16
    )

    # Downsampling with MV2 block.
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=24, strides=2
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )

    # First MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=48, strides=2
    )
    x = mobilevit_block(x, num_blocks=2, projection_dim=64)

    # Second MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=64 * expansion_factor, output_channels=64, strides=2
    )
    x = mobilevit_block(x, num_blocks=4, projection_dim=80)

    # Third MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=80 * expansion_factor, output_channels=80, strides=2
    )
    x = mobilevit_block(x, num_blocks=3, projection_dim=96)
    x = conv_block(x, filters=320, kernel_size=1, strides=1)

    # Classification head.
    x = layers.GlobalAvgPool2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)

mobilevit_xxs = create_mobilevit()
mobilevit_xxs.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 256, 256, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  448         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [5]:
train_dir = "/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/train"
test_dir = "/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/valid"

In [6]:
image_path = "/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed"
for dirpath, dirnames, filenames in os.walk(image_path):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed'.
There are 4 directories and 0 images in '/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/train'.
There are 0 directories and 400 images in '/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/train/cataract'.
There are 0 directories and 400 images in '/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/train/diabetes'.
There are 0 directories and 400 images in '/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/train/amd'.
There are 0 directories and 400 images in '/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/train/normal'.
There are 4 directories and 0 images in '/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/valid'.
There are 0 directories and 100 images in '/content/drive/MyDrive/Eye Disease Classification/DATASET/Preprocessed/valid/amd'.
The

In [7]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="categorical",
                                                                 image_size=(IMG_SIZE,IMG_SIZE))

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=(IMG_SIZE,IMG_SIZE),
                                                                shuffle=False)

Found 1600 files belonging to 4 classes.
Found 400 files belonging to 4 classes.


In [8]:
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing_factor)

def run_experiment(epochs=EPOCHS):
    mobilevit_xxs = create_mobilevit(num_classes=num_classes)
    mobilevit_xxs.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

    checkpoint_filepath = "Fundus_data_model_checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    mobilevit_xxs.fit(
        train_data,
        validation_data=test_data,
        epochs=EPOCHS,
        callbacks=[checkpoint_callback],
    )
    mobilevit_xxs.load_weights(checkpoint_filepath)
    _, accuracy = mobilevit_xxs.evaluate(test_data)
    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")
    return mobilevit_xxs

mobilevit_xxs = run_experiment()

Epoch 1/100
50/50 [==============================] - 553s 8s/step - loss: 1.0988 - accuracy: 0.5475 - val_loss: 1.4255 - val_accuracy: 0.2500
Epoch 2/100
50/50 [==============================] - 39s 721ms/step - loss: 0.9111 - accuracy: 0.6587 - val_loss: 1.4616 - val_accuracy: 0.2500
Epoch 3/100
50/50 [==============================] - 35s 645ms/step - loss: 0.8301 - accuracy: 0.7256 - val_loss: 1.5104 - val_accuracy: 0.2500
Epoch 4/100
50/50 [==============================] - 36s 660ms/step - loss: 0.7249 - accuracy: 0.7819 - val_loss: 1.6707 - val_accuracy: 0.2500
Epoch 5/100
50/50 [==============================] - 34s 610ms/step - loss: 0.5992 - accuracy: 0.8794 - val_loss: 2.1209 - val_accuracy: 0.2500
Epoch 6/100
50/50 [==============================] - 34s 613ms/step - loss: 0.5062 - accuracy: 0.9369 - val_loss: 1.7374 - val_accuracy: 0.2500
Epoch 7/100
50/50 [==============================] - 39s 722ms/step - loss: 0.4539 - accuracy: 0.9631 - val_loss: 2.6196 - val_accuracy: 0

In [9]:
pred_probs = mobilevit_xxs.predict(test_data, verbose=1) # set verbosity to see how long it will take

13/13 [==============================] - 8s 413ms/step


In [10]:
pred_classes = pred_probs.argmax(axis=1)

y_labels = []
for images, labels in test_data.unbatch(): # unbatch the test data and get images and labels
  y_labels.append(labels.numpy().argmax())

In [12]:
target_names = ['amd', 'cataract', 'diabetes','normal']
print(classification_report(y_labels,
                            pred_classes,
                            target_names=target_names, digits=4))

              precision    recall  f1-score   support

         amd     0.8017    0.9300    0.8611       100
    cataract     0.9515    0.9800    0.9655       100
    diabetes     0.7229    0.6000    0.6557       100
      normal     0.7857    0.7700    0.7778       100

    accuracy                         0.8200       400
   macro avg     0.8154    0.8200    0.8150       400
weighted avg     0.8154    0.8200    0.8150       400



In [15]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc

In [16]:
import seaborn as sns
cnf_matrix = confusion_matrix(y_labels,
                              pred_classes)

TP = cnf_matrix[0, 0]
TN = cnf_matrix[1:, 1:].sum()
FP = cnf_matrix[0, 1:].sum()
FN = cnf_matrix[1:, 0].sum()

Population = TN+FN+TP+FP
spesificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)

print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)
print("Spesificity:", spesificity)
print("Sensitivity:", sensitivity)

True Positives: 93
False Positives: 7
True Negatives: 277
False Negatives: 23
Spesificity: 0.9753521126760564
Sensitivity: 0.8017241379310345


In [18]:
# Compute confusion matrix
import seaborn as sns
import numpy as np
cnf_matrix = confusion_matrix(y_labels,
                              pred_classes)
np.set_printoptions(precision=5)

x_axis_labels = ['amd', 'cataract','diabetes', 'normal']
y_axis_labels = ['amd', 'cataract', 'diabetes','normal']
plt.figure(figsize=(1,1), dpi=400)
sns.set(font_scale=0.3)
b = sns.heatmap(cnf_matrix, annot=True, square = True,
            cbar=False, cmap='Greens',
            annot_kws={'size': 3},
            fmt='g',
            xticklabels=x_axis_labels,
            yticklabels=y_axis_labels)